# ETL: автоматизация подготовки данных семинары

## Урок 4. Партицирование данных по дате. Динамическое партицирование

1. Создайте таблицу movies с полями movies_type, director, year_of_issue, length_in_minutes, rate.

2. Сделайте таблицы для горизонтального партицирования по году выпуска (до 1990, 1990 -2000, 2000- 2010, 2010-2020, после 2020).

3. Сделайте таблицы для горизонтального партицирования по длине фильма (до 40 минута, от 40 до 90 минут, от 90 до 130 минут, более 130 минут).

4. Сделайте таблицы для горизонтального партицирования по рейтингу фильма (ниже 5, от 5 до 8, от 8до 10).

5. Создайте правила добавления данных для каждой таблицы.

6. Добавьте фильмы так, чтобы в каждой таблице было не менее 3 фильмов.

7. Добавьте пару фильмов с рейтингом выше 10.

In [1]:
import init_spark_env

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [3]:
# Инициализация Spark Session
spark = SparkSession.builder .appName("Movies Data Preparation").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/16 21:01:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark

In [7]:
# 1. Создаем схему DataFrame для таблицы movies
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

schema = StructType([
    StructField("movies_type", StringType(), True),
    StructField("director", StringType(), True),
    StructField("year_of_issue", IntegerType(), True),
    StructField("length_in_minutes", IntegerType(), True),
    StructField("rate", FloatType(), True)
])

In [12]:
# 2. Создаем DataFrame для партицирования по году выпуска
movies_rdd = spark.sparkContext.parallelize([
    Row(movies_type='Action', director='Director A', year_of_issue=1985, length_in_minutes=120, rate=7.2),
    Row(movies_type='Drama', director='Director B', year_of_issue=1995, length_in_minutes=150, rate=8.5),
    Row(movies_type='Comedy', director='Director C', year_of_issue=2005, length_in_minutes=90, rate=6.5),
])
movies_df = spark.createDataFrame(movies_rdd, schema)

In [15]:
# Горизонтальное партицирование по году выпуска
movies_pre_1990 = movies_df.filter("year_of_issue < 1990")
movies_1990_2000 = movies_df.filter("year_of_issue >= 1990 AND year_of_issue < 2000")
movies_2000_2010 = movies_df.filter("year_of_issue >= 2000 AND year_of_issue < 2010")
movies_2010_2020 = movies_df.filter("year_of_issue >= 2010 AND year_of_issue < 2020")
movies_post_2020 = movies_df.filter("year_of_issue >= 2020")

In [16]:
# 3. Горизонтальное партицирование по длине фильма
movies_short = movies_df.filter("length_in_minutes < 40")
movies_medium = movies_df.filter("length_in_minutes >= 40 AND length_in_minutes < 90")
movies_long = movies_df.filter("length_in_minutes >= 90 AND length_in_minutes < 130")
movies_very_long = movies_df.filter("length_in_minutes >= 130")

In [17]:
# 4. Горизонтальное партицирование по рейтингу фильма
movies_low_rate = movies_df.filter("rate < 5")
movies_medium_rate = movies_df.filter("rate >= 5 AND rate < 8")
movies_high_rate = movies_df.filter("rate >= 8 AND rate <= 10")

In [23]:
# 5. В Apache Spark данные автоматически добавляются в DataFrame при его создании. 
# Но, если мы хотим добавить данные динамически, можно использовать union или join операции.

# 6. Добавление фильмов в DataFrame (для примера добавим три фильма с рейтингом выше 10)
extra_movies_rdd = spark.sparkContext.parallelize([
    Row(movies_type='Sci-Fi', director='Director X', year_of_issue=2021, length_in_minutes=140, rate=10.5),
    Row(movies_type='Fantasy', director='Director Y', year_of_issue=2022, length_in_minutes=200, rate=10.8),
    Row(movies_type='Horror', director='Director Z', year_of_issue=2023, length_in_minutes=80, rate=11.0),
])
extra_movies_df = spark.createDataFrame(extra_movies_rdd, schema)
movies_df = movies_df.union(extra_movies_df)

In [24]:
# 7. Фильмы с рейтингом выше 10 уже добавлены в предыдущем шаге.

# 8. Выбор из всех таблиц
movies_pre_1990.show()
movies_1990_2000.show()
movies_2000_2010.show()
movies_2010_2020.show()
movies_post_2020.show()
movies_short.show()
movies_medium.show()
movies_long.show()
movies_very_long.show()
movies_low_rate.show()
movies_medium_rate.show()
movies_high_rate.show()

+-----------+----------+-------------+-----------------+----+
|movies_type|  director|year_of_issue|length_in_minutes|rate|
+-----------+----------+-------------+-----------------+----+
|     Action|Director A|         1985|              120| 7.2|
+-----------+----------+-------------+-----------------+----+

+-----------+----------+-------------+-----------------+----+
|movies_type|  director|year_of_issue|length_in_minutes|rate|
+-----------+----------+-------------+-----------------+----+
|      Drama|Director B|         1995|              150| 8.5|
+-----------+----------+-------------+-----------------+----+

+-----------+----------+-------------+-----------------+----+
|movies_type|  director|year_of_issue|length_in_minutes|rate|
+-----------+----------+-------------+-----------------+----+
|     Comedy|Director C|         2005|               90| 6.5|
+-----------+----------+-------------+-----------------+----+

+-----------+--------+-------------+-----------------+----+
|movies

In [25]:
# 9. Выбор только из основной таблицы
movies_df.show()

+-----------+----------+-------------+-----------------+----+
|movies_type|  director|year_of_issue|length_in_minutes|rate|
+-----------+----------+-------------+-----------------+----+
|     Action|Director A|         1985|              120| 7.2|
|      Drama|Director B|         1995|              150| 8.5|
|     Comedy|Director C|         2005|               90| 6.5|
|     Sci-Fi|Director X|         2021|              140|10.5|
|    Fantasy|Director Y|         2022|              200|10.8|
|     Horror|Director Z|         2023|               80|11.0|
|     Sci-Fi|Director X|         2021|              140|10.5|
|    Fantasy|Director Y|         2022|              200|10.8|
|     Horror|Director Z|         2023|               80|11.0|
|     Sci-Fi|Director X|         2021|              140|10.5|
|    Fantasy|Director Y|         2022|              200|10.8|
|     Horror|Director Z|         2023|               80|11.0|
|     Sci-Fi|Director X|         2021|              140|10.5|
|    Fan

In [26]:
# Закрываем Spark Session после работы
spark.stop()